# Installation des dépendances et Imports
Librairies nécessaires (Transformers, Faker, Accelerate).

In [ ]:

!pip install -q git+https://github.com/huggingface/transformers
!pip install -q accelerate bitsandbytes faker

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from faker import Faker
import re
import hashlib
import random
import json
from datetime import datetime

# Configuration du device (GPU recommandé)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f" Environnement prêt. Exécution sur : {device}")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
 Environnement prêt. Exécution sur : cuda


 # Génération de Données : Documents Synthétiques (Faker)
 Nous utilisons Faker pour générer une note fiscale factice contenant des PII (Informations Identifiables Personnellement).


In [ ]:
fake = Faker('fr_FR')

def generate_fake_tax_document():
    """Génère un document fiscal factice avec des entités nommées."""

    # Génération d'entités
    nom_personne = fake.name()
    entreprise = fake.company()
    siret = fake.siret().replace(" ", "")
    adresse = fake.address().replace("\n", ", ")
    iban = fake.iban()
    age = random.randint(25, 65)
    revenu = random.randint(30000, 80000)
    impot = int(revenu * 0.15)
    date_doc = fake.date_this_year().strftime("%d/%m/%Y")

    # Modèle de texte (Note fiscale)
    document = f"""
    RÉPUBLIQUE FRANÇAISE - AVIS D'IMPÔT {datetime.now().year}

    DESTINATAIRE :
    Monsieur/Madame {nom_personne}
    Né(e) le : {fake.date_of_birth(minimum_age=age, maximum_age=age)} ({age} ans)
    Adresse : {adresse}

    RÉFÉRENCES :
    SIRET de l'employeur principal : {siret} ({entreprise})
    Numéro Fiscal : {random.randint(1000000000000, 9999999999999)}

    DÉTAILS FINANCIERS :
    Revenu Brut Global : {revenu} €
    Montant de l'impôt à payer : {impot} €

    PAIEMENT :
    Prélèvement automatique sur le compte {iban}
    Date limite : {date_doc}
    """
    return document, {"nom": nom_personne, "org": entreprise, "age": age, "revenu": revenu, "iban": iban}

# Générer et afficher un exemple
doc_original, entites_verite = generate_fake_tax_document()
print("--- DOCUMENT ORIGINAL GÉNÉRÉ ---")
print(doc_original)

--- DOCUMENT ORIGINAL GÉNÉRÉ ---

    RÉPUBLIQUE FRANÇAISE - AVIS D'IMPÔT 2026

    DESTINATAIRE :
    Monsieur/Madame Olivier Louis
    Né(e) le : 1984-08-24 (41 ans)
    Adresse : 69, avenue Jean Guillou, 72159 Saint Juliette

    RÉFÉRENCES :
    SIRET de l'employeur principal : 33543975800095 (Delattre Martineau SARL)
    Numéro Fiscal : 6942570143058

    DÉTAILS FINANCIERS :
    Revenu Brut Global : 54669 €
    Montant de l'impôt à payer : 8200 €

    PAIEMENT :
    Prélèvement automatique sur le compte FR1910436626999571265498872
    Date limite : 02/01/2026
    


In [ ]:
!pip install --upgrade transformers accelerate bitsandbytes

# Chargement du Modèle Phi-3 Mini

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_id = "microsoft/Phi-3-mini-4k-instruct"

print(" Chargement du tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

print(" Chargement du modèle (Version Native)...")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=False,
    attn_implementation="eager"
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

print(" Modèle Phi-3 Mini chargé avec succès !")

# Test rapide pour vérifier que ça marche
test_input = [{"role": "user", "content": "Bonjour, écris une phrase de test."}]
print(pipe(test_input, max_new_tokens=20)[0]['generated_text'])

 Chargement du tokenizer...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


 Chargement du modèle (Version Native)...


Loading weights:   0%|          | 0/195 [00:00<?, ?it/s]

 Modèle Phi-3 Mini chargé avec succès !


Both `max_new_tokens` (=20) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[{'role': 'user', 'content': 'Bonjour, écris une phrase de test.'}, {'role': 'assistant', 'content': ' "Bonjour, je voudrais tester la fonctionnalité de prise en'}]


# Instructions & Inférence (Le Cœur de l'Anonymisation)
Nous définissons le **System Prompt** pour instruire le modèle sur les règles de remplacement.


In [ ]:
# Définition des règles pour le modèle
system_prompt = """Vous êtes un assistant expert en protection des données et anonymisation (RGPD).
Votre tâche est de réécrire le document fiscal fourni en appliquant STRICTEMENT les règles suivantes :

1. Noms de personnes : Remplacer par [PERSONNE_1], [PERSONNE_2]...
2. Entreprises : Remplacer par [ENTREPRISE_1]...
3. Adresses postales complètes : Remplacer par [ADRESSE_SUPPRIMEE]
4. Identifiants (IBAN, SIRET, Numéro Fiscal) : Remplacer par [SUPPRIME]
5. Âges : Généraliser par une tranche de 10 ans (ex: 35 ans -> [30-40 ans]).
6. Montants financiers : Remplacer par une tranche (Banding) (ex: 47 523€ -> [40k-50k€]).

Ne changez pas la structure du document. Ne conservez AUCUNE donnée sensible réelle.
"""

# Construction de la conversation
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": f"Voici le document à anonymiser :\n\n{doc_original}"}
]

# Paramètres de génération
generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.1, # Température basse pour être déterministe et suivre les règles
    "do_sample": True,
}

# Inférence
print(" Anonymisation en cours avec Phi-3...")
output = pipe(messages, **generation_args)
doc_anonymise_llm = output[0]['generated_text']

print("\n--- RÉSULTAT DU MODÈLE (Inférence) ---")
print(doc_anonymise_llm)

Both `max_new_tokens` (=500) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Anonymisation en cours avec Phi-3...

--- RÉSULTAT DU MODÈLE (Inférence) ---
 Voici le document anonymisé :


    RÉPUBLIQUE FRANÇAISE - AVIS D'IMPÔT 2026

    DESTINATAIRE :
    [PERSONNE_1]
    Né(e) le : [30-40 ans] (41 ans)
    Adresse : [ADRESSE_SUPPRIMEE]

    RÉFÉRENCES :
    SIRET de l'employeur principal : [SUPPRIME]
    Numéro Fiscal : [SUPPRIME]

    DÉTAILS FINANCIERS :
    Revenu Brut Global : [40k-50k€]
    Montant de l'impôt à payer : [40k-50k€]

    PAIEMENT :
    Prélèvement automatique sur le compte [SUPPRIME]
    Date limite : 02/01/2026


#  Fonctions Python Complémentaires (HMAC & Banding Avancé)

 Pour les calculs mathématiques  (Hashing SHA256, Banding exact), il est préférable d'utiliser le LLM pour *taguer* les entités, puis Python pour calculer.


In [ ]:

def hmac_sha256_anonymizer(valeur, secret_key="mon_secret_fiscal"):
    """Hache une valeur (âge, id) avec une clé secrète."""
    message = str(valeur).encode()
    key = secret_key.encode()
    return hashlib.sha256(key + message).hexdigest()[:10] # On garde les 10 premiers caractères

def financial_banding(montant):
    """Transforme un montant exact en tranche."""
    # Arrondi au 10k le plus proche pour l'exemple
    lower = (montant // 10000) * 10000
    upper = lower + 10000
    return f"[{lower/1000}k-{upper/1000}k€]"

print("--- DÉMONSTRATION PYTHON (Règles strictes) ---")
print(f"Âge original ({entites_verite['age']}) -> Hash HMAC : {hmac_sha256_anonymizer(entites_verite['age'])}")
print(f"Revenu original ({entites_verite['revenu']}€) -> Banding : {financial_banding(entites_verite['revenu'])}")

--- DÉMONSTRATION PYTHON (Règles strictes) ---
Âge original (41) -> Hash HMAC : 9049a53264
Revenu original (54669€) -> Banding : [50.0k-60.0k€]


#  Validation (Contrôle Regex)
 Un script de sécurité vérifie qu'aucun format sensible n'a "fuité" dans la sortie du modèle.


In [ ]:
def validation_securite(texte):
    alertes = []

    # Regex Patterns pour la France
    patterns = {
        "IBAN_FR": r"FR\d{2}[ ]?\d{4}[ ]?\d{4}[ ]?\d{4}[ ]?\d{4}[ ]?\d{2}",
        "SIRET": r"\d{14}",
        "MONTANT_EXACT": r"\d{2,3}\s?\d{3}\s?€", # Détecte des montants comme 45 000 € non floutés
        "EMAIL": r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"
    }

    score_securite = 100

    for type_donnee, pattern in patterns.items():
        matches = re.findall(pattern, texte)
        if matches:
            alertes.append(f" ALERTE : {len(matches)} {type_donnee} détecté(s) ! (ex: {matches[0]})")
            score_securite -= 25

    if not alertes:
        return " Validation RÉUSSIE. Aucune donnée sensible détectée par Regex.", 100
    else:
        return "\n".join(alertes), score_securite

# Lancer la validation sur le texte produit par Phi-3
rapport, score = validation_securite(doc_anonymise_llm)

print("--- RAPPORT DE VALIDATION ---")
print(rapport)
print(f"Score de conformité : {score}/100")

--- RAPPORT DE VALIDATION ---
 Validation RÉUSSIE. Aucune donnée sensible détectée par Regex.
Score de conformité : 100/100


In [ ]:

def calculer_score_confidentialite(texte_original, entites_reelles, texte_anonymise):
    """
    Calcule si les valeurs réelles (Faker) sont encore présentes dans le texte anonymisé.
    TP (Vrai Positif) = Donnée sensible correctement supprimée.
    FN (Faux Négatif) = Donnée sensible encore présente (Fuite/Leak).
    """
    stats = {
        "total_secrets": 0,
        "secrets_supprimes": 0,
        "fuites": []
    }

    # Liste des valeurs à vérifier (Nom, IBAN, SIRET, etc.)
    valeurs_a_cacher = [
        str(entites_reelles['nom']),
        str(entites_reelles['iban']),
        str(entites_reelles['org']),
        str(entites_reelles['revenu']), # Le montant exact doit disparaître
        str(entites_reelles['siret']) if 'siret' in entites_reelles else None
    ]

    # Nettoyage des valeurs None
    valeurs_a_cacher = [v for v in valeurs_a_cacher if v]

    stats["total_secrets"] = len(valeurs_a_cacher)

    print("--- DÉTAIL DU SCAN ---")
    for secret in valeurs_a_cacher:
        # On vérifie si le secret est dans le texte (insensible à la casse)
        if secret.lower() in texte_anonymise.lower():
            print(f" FUITE DÉTECTÉE : '{secret}' est encore visible !")
            stats["fuites"].append(secret)
        else:
            print(f" PROTEGÉ : '{secret}' a bien disparu.")
            stats["secrets_supprimes"] += 1

    # Calcul du Recall (Rappel)
    recall = (stats["secrets_supprimes"] / stats["total_secrets"]) * 100 if stats["total_secrets"] > 0 else 0

    return recall, stats


if 'entites_verite' in globals() and 'doc_anonymise_llm' in globals():
    score, details = calculer_score_confidentialite(doc_original, entites_verite, doc_anonymise_llm)

    print("\n" + "="*30)
    print(f" SCORE DE CONFIDENTIALITÉ (Recall) : {score:.2f}%")
    print("="*30)

    if score == 100:
        print("Excellent ! Toutes les données identifiées ont été masquées/transformées.")
    else:
        print(f"Attention, il reste {len(details['fuites'])} données sensibles.")
else:
    print("Erreur : Veuillez d'abord lancer la génération Faker et l'inférence Phi-3 (étapes 2 et 4).")

--- DÉTAIL DU SCAN ---
 PROTEGÉ : 'Olivier Louis' a bien disparu.
 PROTEGÉ : 'FR1910436626999571265498872' a bien disparu.
 PROTEGÉ : 'Delattre Martineau SARL' a bien disparu.
 PROTEGÉ : '54669' a bien disparu.

 SCORE DE CONFIDENTIALITÉ (Recall) : 100.00%
Excellent ! Toutes les données identifiées ont été masquées/transformées.


In [ ]:
from google.colab import drive
import os

# Cela va monter votre Drive dans le dossier /content/drive
drive.mount('/content/drive')

print(" Google Drive connecté !")

Mounted at /content/drive
 Google Drive connecté !


In [ ]:
# @title 4. Anonymisation (Code Robuste & Sécurisé)
import torch

# 1. Préparation propre des inputs
# On s'assure que le modèle et les inputs sont sur le même device (GPU)
device = model.device

# Création du prompt
messages_perso = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": f"Voici le document à anonymiser :\n\n{texte_extrait}"}
]

# 2. Tokenization sécurisée
# On génère les ID de tokens
input_ids = tokenizer.apply_chat_template(
    messages_perso,
    return_tensors="pt",
    add_generation_prompt=True
).to(device)

# 3. Vérification de la longueur (Sécurité context window)
# Phi-3-mini supporte 4096 tokens. Si on dépasse, ça plante.
longueur_input = input_ids.shape[1]
print(f" Longueur de l'entrée : {longueur_input} tokens (Max: 4096)")

if longueur_input > 3500:
    print(" ATTENTION : Document très long. Je ne garde qu'une partie.")
    # On garde le system prompt (début) et la fin du document
    pass

# 4. Lancement de l'Inférence
print("\n Démarrage de l'IA...")
print("-" * 50)

# On redéfinit le streamer au cas où
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

# CORRECTION MAJEURE ICI : On passe explicitement 'input_ids' par mot-clé
try:
    outputs = model.generate(
        input_ids=input_ids,
        streamer=streamer,
        max_new_tokens=1500,
        temperature=0.1,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id
    )
except Exception as e:
    print(f"\n❌ Erreur pendant la génération : {e}")
    print("Conseil : Essayez de réduire la taille du document ou de redémarrer le runtime.")

AttributeError: 